# <font size=35 color=lightgreen>** Sentiment API **<font>🥲
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [2]:

import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import re
import string
import chardet
import sklearn
import fastapi
import joblib
import nltk
import unicodedata
import urllib.request
from io import StringIO
import urllib.response
import os
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")





### <font size = 8 color="lightgreen">Importación de los datasets<font>

#### **Función importación dataset**

In [3]:
def importar_dataset(url, separator=';'):
    """
    Importa dataset desde URL detectando encoding automáticamente.
    """
    try:
        # 1. Descargar contenido una sola vez
        with urllib.request.urlopen(url) as response:
            content = response.read()

        # 2. Detectar encoding
        result = chardet.detect(content)
        encoding = result['encoding']
        print(f"🔍 Encoding detectado: {encoding} (confianza: {result['confidence']:.2%})")

        # 3. Decodificar y cargar en DataFrame
        decoded_content = content.decode(encoding, errors='replace')
        data = pd.read_csv(StringIO(decoded_content), sep=separator)

        print("✅ Archivo cargado correctamente")
        print(f"📊 Tamaño del dataset: {data.shape}")
        print("\n🔍 Muestra aleatoria (3 registros):")
        print(data.sample(3))

        return data

    except urllib.error.URLError as e:
        print(f"❌ Error de URL: {e}")
        return None
    except pd.errors.ParserError as e:
        print(f"❌ Error al parsear CSV: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {type(e).__name__}: {e}")
        return None

#### **Dataset1: sentimentdataset_es.csv**

In [4]:
df1_raw = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/sentimentdataset_es.csv")


🔍 Encoding detectado: Windows-1252 (confianza: 72.97%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (732, 15)

🔍 Muestra aleatoria (3 registros):
     Unnamed: 0.1  Unnamed: 0  \
304            71          73   
386           371         375   
538           659         663   

                                                  Text      Sentiment  \
304   Encontrar la calma en medio de un día ajetreado.        Neutral   
386  Hipnotizado por la danza cósmica de luciérnaga...  Encantamiento   
538  Organizar una fiesta de pijamas con amigos est...     Excitación   

            Timestamp                     User   Platform  \
304   18-02-2023 9:30                ZenMaster    Twitter   
386  10-08-2022 22:00         NightSkyObserver  Instagram   
538  17-08-2023 22:15  SleepoverHostHighSchool    Twitter   

                                              Hashtags  Retweets  Likes  \
304                                 #Calma #MenteEnPaz        15     30   
386               #Encan

#### **Dataset2: sentiment_analysis_dataset.csv**

In [5]:
df2_raw = importar_dataset("https://raw.githubusercontent.com/ml-punto-tech/sentiment-api/refs/heads/feature/data-science-marely/data-science/datasets/datasets-origin/sentiment_analysis_dataset.csv")

🔍 Encoding detectado: Windows-1252 (confianza: 73.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (2540, 3)

🔍 Muestra aleatoria (3 registros):
                                                  texto  label sentimiento
2272  ¿Por qué mierda de todos los personajes y de t...      2    positivo
2025  Nada mas satisfactorio que darse cuenta de tod...      1     neutral
1765  Busco una calma inalcanzable La atmósfera aquí...      2    positivo


In [6]:
df3_raw = importar_dataset("https://github.com/eduardotec05/datasets/raw/refs/heads/main/twitter_training_esp_convertido%20(2).csv", separator=',')

🔍 Encoding detectado: UTF-8-SIG (confianza: 100.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (74682, 4)

🔍 Muestra aleatoria (3 registros):
          id plataforma sentimiento  \
9134    9569  Overwatch     Neutral   
4982      56     Amazon     Neutral   
46323  11949    Verizon    Negativo   

                                                   texto  
9134   gente normal: f pagar a los cosmonautas humano...  
4982   Lo último que necesitan es ir a trabajar a dis...  
46323  Es muy solitario estar en la cima porque no ha...  


In [7]:
df3_raw.head()

,id,plataforma,sentimiento,texto
0,2401,Borderlands,Positivo,Estoy llegando a Borderlands y los asesinaré a...
1,2401,Borderlands,Positivo,"Voy a llegar a las fronteras y os mataré a todos,"
2,2401,Borderlands,Positivo,Voy a llegar a Borderlands y los mataré a todos.
3,2401,Borderlands,Positivo,Voy a llegar a Borderlands y los asesinaré a t...
4,2401,Borderlands,Positivo,Me estoy metiendo en Borderlands 2 y os voy a ...


In [8]:
def verificar_calidad_importacion(df, nombre_dataset):
    """
    Verifica que no se haya perdido información durante la importación.
    """
    print(f"\n🔍 VERIFICACIÓN DE CALIDAD: {nombre_dataset}")
    print("=" * 60)

    if df is None:
        print("❌ Dataset es None")
        return False

    # 1. Información básica
    print(f"📊 Forma: {df.shape}")
    print(f"📝 Columnas: {list(df.columns)}")

    # 2. Buscar columnas de texto
    columnas_texto = [col for col in df.columns if df[col].dtype == 'object']
    print(f"🔤 Columnas de texto: {columnas_texto}")

    if not columnas_texto:
        print("⚠️  No se encontraron columnas de texto")
        return True

    # 3. Analizar una columna de texto (usar la primera)
    col_texto = columnas_texto[0]
    print(f"\n📝 Analizando columna: '{col_texto}'")

    # Muestra de textos
    textos = df[col_texto].dropna().head(5).tolist()

    problemas = []

    for i, texto in enumerate(textos):
        if isinstance(texto, str):
            # Buscar caracteres de reemplazo (�) que indican problemas
            caracteres_problema = texto.count('�')
            if caracteres_problema > 0:
                problemas.append(f"Texto {i+1} tiene {caracteres_problema} caracteres de reemplazo (�)")

            # Buscar emojis
            emojis = [c for c in texto if unicodedata.category(c)[0] in ['S', 'So']]
            if emojis:
                print(f"  Texto {i+1}: ✅ Tiene {len(emojis)} emoji(s): {''.join(emojis[:3])}")
            else:
                print(f"  Texto {i+1}: 📄 Sin emojis")

            # Mostrar fragmento
            preview = texto[:80] + "..." if len(texto) > 80 else texto
            print(f"     '{preview}'")

    # 4. Resumen
    if problemas:
        print(f"\n⚠️  PROBLEMAS ENCONTRADOS:")
        for problema in problemas:
            print(f"   • {problema}")
        return False
    else:
        print(f"\n✅ CALIDAD OK: No se detectaron caracteres perdidos")
        return True

verificar_calidad_importacion(df1_raw, "Dataset 1")
verificar_calidad_importacion(df2_raw, "Dataset 2")
verificar_calidad_importacion(df3_raw, "Dataset 3")


🔍 VERIFICACIÓN DE CALIDAD: Dataset 1
📊 Forma: (732, 15)
📝 Columnas: ['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour']
🔤 Columnas de texto: ['Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Country']

📝 Analizando columna: 'Text'
  Texto 1: 📄 Sin emojis
     '¡Acabo de adoptar a un lindo amigo peludo!??'
  Texto 2: 📄 Sin emojis
     '¡Acabo de terminar un entrenamiento increíble!??'
  Texto 3: 📄 Sin emojis
     '¡Adoración desbordante por un lindo cachorro rescatado!??'
  Texto 4: 📄 Sin emojis
     '¡Año nuevo, nuevos objetivos de fitness!??'
  Texto 5: 📄 Sin emojis
     '¡Celebrando el cumpleaños de un amigo esta noche!??'

✅ CALIDAD OK: No se detectaron caracteres perdidos

🔍 VERIFICACIÓN DE CALIDAD: Dataset 2
📊 Forma: (2540, 3)
📝 Columnas: ['texto', 'label', 'sentimiento']
🔤 Columnas de texto: ['texto', 'sentimiento']

📝 Analizando columna: 'texto'
  Texto

True

<font color='lightgreen' size=12>Filtrar datasets</font>

In [9]:
# Función filtrar dataset
def filtrar_dataset(data):
    data_filtro = data[['texto', 'sentimiento']]
    data_filtro = data_filtro[data_filtro['texto'].str.strip() != ""]
    print(data_filtro.sample(5))
    return data_filtro

# Reemplazar nombre columnas Text por texto, Sentiment por sentimiento
df1_raw.rename({'Text':'texto', 'Sentiment':'sentimiento'}, axis=1, inplace=True)
df1_filtrado = filtrar_dataset(df1_raw)
df2_filtrado = filtrar_dataset(df2_raw)
df3_filtrado = filtrar_dataset(df3_raw)

                                                 texto sentimiento
96   Asistió a un concierto de jazz y se balanceó a...     Alegría
30   Abrazar la belleza de la naturaleza, un moment...    Positivo
412  Intentó un truco de magia para impresionar a s...   Verguenza
242  Embárcate en una odisea culinaria, saboreando ...     Neutral
42   Accidentalmente me gustó la foto antigua de la...   Verguenza
                                                  texto sentimiento
2468  mira que te voy a patear deja de hacerte el lo...    positivo
2002  Risto Mejide es tan sOlo un pobre hombre con í...     neutral
360   #gelp Son un fracaso total Cowen. Vos y los ap...    negativo
481   Mañana viajo muy lejos solo por primera vez, e...    negativo
2358  Que esperanzador y digno tener a la ministra @...    positivo
                                                   texto sentimiento
37272  Estoy impresionado con el nivel de documentaci...    Positivo
3358   Por favor, sé falso. Se parece demasiado a cu

### <font size= 12 color="lightgreen" >Explorando los datasets<font>

In [10]:
# Crear función para explorar datasets
def explorar_dataset(data):
    print('Filas: ' + str(data.shape[0]))
    print('Columnas: ' + str(data.shape[1]))
    print('\nColumnas: \n' + str(data.columns.tolist()))
    print('\nTipo de datos: \n' + str(data.dtypes))
    print('\nValores nulos: \n' + str(data.isnull().sum()))
    print('\nMuestra aleatoria (5 registros): \n' + str(data.sample(5)))

#### **Explorando Data1**

In [11]:
explorar_dataset(df1_filtrado)

Filas: 732
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto  sentimiento
196  Destrozada por la traición, la confianza se de...     Traición
383  Gratitud por las pequeñas alegrías que trae ca...     Gratitud
158  Con el corazón entusiasmado, corriendo por cam...        Ánimo
591  Rebosante de alegría, una taza de risa compart...      Alegría
344  Evitando los fragmentos de sueños destrozados,...  Resiliencia


#### **Explorando data2**

In [12]:
explorar_dataset(df1_filtrado)

Filas: 732
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
473  La soledad, una compañera silenciosa en la noc...     Neutral
6    ¡Compartiendo amor y vibraciones positivas con...        Amor
182  Debajo de las luces de la ciudad, la bailarina...    Positivo
292  En los Oscar, el actor acepta gentilmente un p...    Gratitud
146  Comenzó a aprender bailes de salón, deslizándo...     Alegría


In [13]:
explorar_dataset(df3_filtrado)

Filas: 74682
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          41
sentimiento     0
dtype: int64

Muestra aleatoria (5 registros): 
                                                   texto  sentimiento
63171  RhandlerR... ¡Vaya! No puedo jugar por este er...     Negativo
40484       ¡Mira mi transmisión desde mi PlayStation 4!  Irrelevante
51939  He estado jugando a Red Headed Redemption dura...     Positivo
44962      W Esto es ridículo pic.twitter.com/4TNeHzobjS     Negativo
24240  Comprar reseÃ±as negativas de Google - ReseÃ±a...      Neutral


### <font size=12 color=lightgreen>Limpiar textos</font>

#### **Función para limpieza de textos**

In [14]:
def limpiar_texto_sentimientos(texto):
    """
    Normaliza texto español preservando ñ y eliminando tildes.
    NO convierte a minúsculas para preservar intensidad emocional.
    """
    # Verifica si la entrada no es una cadena. Si no lo es, devuelve una cadena vacía.
    if not isinstance(texto, str):
        return ""

    # 1. Normaliza el texto para separar los caracteres base de sus diacríticos (ej., tildes).
    texto = unicodedata.normalize('NFD', texto)

    # 2. Reemplaza temporalmente las 'ñ' y 'Ñ' con marcadores especiales para preservarlas
    # durante la eliminación de diacríticos.
    texto = texto.replace('n\u0303', '@@@N_TILDE@@@')
    texto = texto.replace('ñ', '@@@N_TILDE@@@')
    texto = texto.replace('N\u0303', '@@@N_TILDE_MAYUS@@@')
    texto = texto.replace('Ñ', '@@@N_TILDE_MAYUS@@@')

    # 3. Elimina los caracteres diacríticos (como las tildes) del texto.
    texto = ''.join(
        char for char in texto
        if not unicodedata.combining(char)
    )

    # Restaura las 'ñ' y 'Ñ' utilizando los marcadores temporales.
    texto = texto.replace('@@@N_TILDE@@@', 'ñ')
    texto = texto.replace('@@@N_TILDE_MAYUS@@@', 'Ñ')

    # Variable para almacenar el resultado de la limpieza.
    resultado = texto
    chars = []

    # Itera sobre cada caracter en el resultado y añade solo los caracteres imprimibles a una lista.
    # Los caracteres no imprimibles (como los de control) son reemplazados por un espacio.
    for char in resultado:
        if char.isprintable():
            chars.append(char)
        else:
            chars.append(' ')
    resultado = ''.join(chars)

    # Elimina URLs que terminan en "..." (posibles URLs rotas).
    resultado = re.sub(r'https?://[^\s]*\.\.\.', '[URL_ROTA]', resultado)
    resultado = re.sub(r'www\.[^\s]*\\.\\.\\.', '[URL_ROTA]', resultado)

    # Normaliza los espacios múltiples a uno solo y elimina espacios al inicio y final.
    resultado = ' '.join(resultado.split())
    resultado = resultado.strip()

    # Mostrar resultados estadísticos de la limpieza.




    # Devuelve el texto preprocesado.
    return resultado


#### **Análisis proceso de limpieza de textos**

In [15]:
def analizar_limpieza_sentimientos(df_antes, df_despues, nombre):
    """
    Análisis específico para tu función limpiar_texto_para_sentimientos
    """
    print(f"\n🔍 ANÁLISIS ESPECÍFICO: {nombre}")
    print("="*60)

    # 1. Cambios en caracteres específicos del español
    cambios_especificos = {
        'tildes_eliminadas': 0,
        'ñ_preservadas': 0,
        'urls_eliminadas': 0,
        'mayusculas_preservadas': 0
    }

    # Muestra de 50 textos para análisis detallado
    muestra = min(50, len(df_antes))

    for i in range(muestra):
        if i < len(df_despues):
            texto_antes = str(df_antes.iloc[i]['texto'])
            texto_despues = str(df_despues.iloc[i]['texto'])

            # Contar ñ preservadas
            if 'ñ' in texto_antes.lower() and 'ñ' in texto_despues.lower():
                cambios_especificos['ñ_preservadas'] += 1

            # Contar URLs eliminadas
            import re
            urls_antes = len(re.findall(r'https?://\S+', texto_antes))
            urls_despues = len(re.findall(r'https?://\S+', texto_despues))
            if urls_antes > urls_despues:
                cambios_especificos['urls_eliminadas'] += (urls_antes - urls_despues)

            # Verificar mayúsculas preservadas
            mayus_antes = sum(1 for c in texto_antes if c.isupper())
            mayus_despues = sum(1 for c in texto_despues if c.isupper())
            if mayus_antes > 0 and mayus_despues > 0:
                cambios_especificos['mayusculas_preservadas'] += 1

    print("📊 Cambios específicos de tu limpiador:")
    for cambio, cantidad in cambios_especificos.items():
        print(f"   • {cambio.replace('_', ' ').title()}: {cantidad} de {muestra} textos")



    print("="*60)

In [16]:
# Lista de dataframes para procesar
dataframes = [
    (df1_filtrado, "Dataset 1"),
    (df2_filtrado, "Dataset 2"),
    (df3_filtrado, "Dataset 3")
]

resultados = {}

for df, nombre in dataframes:
    # Aplicar limpieza
    df['Texto_Limpio'] = df['texto'].apply(limpiar_texto_sentimientos)

    # Guardar copia limpia
    resultados[nombre] = df.copy()

    # Mostrar info
    print(f"\n📁 {nombre}")
    print(f"   Registros: {len(df):,}")
    print(f"   Muestra (3 textos):")
    print(df[['texto', 'Texto_Limpio']].sample(3))

# Asignar a variables originales
df1_clean = resultados["Dataset 1"]
df2_clean = resultados["Dataset 2"]
df3_clean = resultados["Dataset 3"]

analizar_limpieza_sentimientos(df1_filtrado, df1_clean, "Dataset 1")
analizar_limpieza_sentimientos(df2_filtrado, df2_clean, "Dataset 2")
analizar_limpieza_sentimientos(df3_filtrado, df3_clean, "Dataset 3")


📁 Dataset 1
   Registros: 732
   Muestra (3 textos):
                                                 texto  \
696  Un entumecimiento impenetrable me protege de l...   
222  El complejo rompecabezas de la vida me deja en...   
270  En el mosh pit de un concierto de Metallica, l...   

                                          Texto_Limpio  
696  Un entumecimiento impenetrable me protege de l...  
222  El complejo rompecabezas de la vida me deja en...  
270  En el mosh pit de un concierto de Metallica, l...  

📁 Dataset 2
   Registros: 2,540
   Muestra (3 textos):
                                                  texto  \
1164  TENEMOS UN PRESIDENTE QUE ESTA MAS ATENTO DE H...   
1280  Buena conferencia de nuestro DT, muy centrado,...   
1351  La calma que ha rozado se ha pausado por el karma   

                                           Texto_Limpio  
1164  TENEMOS UN PRESIDENTE QUE ESTA MAS ATENTO DE H...  
1280  Buena conferencia de nuestro DT, muy centrado,...  
1351  La calma que

In [17]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
666,Sumergirse con entusiasmo en un nuevo proyecto.,Entusiasmo,Sumergirse con entusiasmo en un nuevo proyecto.
209,"Disfrutando del brillo dorado de la alegría, u...",Neutral,"Disfrutando del brillo dorado de la alegria, u..."
164,Contemplación reflexiva en medio de las ruinas...,Reflexión,Contemplacion reflexiva en medio de las ruinas...


In [18]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
1139,"Sin la siguiente estrofa, Pobre Soñador del Tr...",negativo,"Sin la siguiente estrofa, Pobre Soñador del Tr..."
1668,t amo atsumu siempre vas a ser mi nene mi bebé...,positivo,t amo atsumu siempre vas a ser mi nene mi bebe...
803,"Yo de veras estoy loca, me siento sola otra ve...",negativo,"Yo de veras estoy loca, me siento sola otra ve..."


In [19]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
38429,tener,Negativo,tener
24876,"""No tengo palabras para describir la increÃ­bl...",Neutral,"""No tengo palabras para describir la increA bl..."
51608,Así es como se ve la escena inicial de Red Dea...,Neutral,Asi es como se ve la escena inicial de Red Dea...


### <font size=12 color=lightgreen>Categorizar de sentimientos </font>

#### **Limpieza de Sentimientos**

In [20]:
# Función simple para limpiar (igual que usarás después)
def limpiar_sentimiento_simple(sentimiento):
    """Convierte a minúsculas y quita espacios extras."""
    return ' '.join(str(sentimiento).lower().strip().split())

# 1. Obtener sentimientos únicos de ambos datasets
sentimientos_unicos = sorted(list(df1_clean['sentimiento'].unique()) + list(df2_clean['sentimiento'].unique()) + list(df3_clean['sentimiento'].unique()))

print(f"📊 Sentimientos únicos (sin limpiar): {len(sentimientos_unicos)}")
print(f"Muestra (primeros 10): {sentimientos_unicos[:10]}")

# 2. Limpiar la lista de sentimientos únicos
sentimientos_unicos_limpios = [limpiar_sentimiento_simple(s) for s in sentimientos_unicos]

# 3. Eliminar duplicados que aparezcan después de limpiar
sentimientos_unicos_limpios = sorted(set(sentimientos_unicos_limpios))

print(f"\n📊 Sentimientos únicos (limpios): {len(sentimientos_unicos_limpios)}")
print(f"Muestra (primeros 10): {sentimientos_unicos_limpios[:10]}")


📊 Sentimientos únicos (sin limpiar): 113
Muestra (primeros 10): ['Abrumado', 'Aburrimiento', 'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Aislamiento', 'Alegría', 'Amabilidad', 'Amargura']

📊 Sentimientos únicos (limpios): 107
Muestra (primeros 10): ['abrumado', 'aburrimiento', 'aceptación', 'admiración', 'adoración', 'agradecido', 'aislamiento', 'alegría', 'amabilidad', 'amargura']


#### **Categorías Sentimientos**

In [21]:
# 1. Definimos las listas de sentimientos según su categoría
print(f"Total de sentimientos únicos: {len(sentimientos_unicos_limpios)}")
print(sentimientos_unicos_limpios)

# 2. SENTIMIENTOS POSITIVOS COMPLETOS (Bienestar, éxito, alegría, admiración)
positivos = [
    'aceptación', 'admiración', 'adoración', 'agradecido', 'alegría', 'amabilidad', 'amor', 'amistad', 'apreciación', 'armonía', 'asombro', 'cautivación', 'celebración', 'colorido', 'confiado','confianza', 'contentamiento', 'creatividad', 'cumplimiento', 'descubrimiento', 'deslumbrar', 'determinación', 'disfrute','diversión', 'elegancia', 'emoción', 'empático', 'empoderamiento',
    'encantamiento', 'energía', 'entusiasmo', 'esperanza', 'euforia', 'excitación', 'felicidad', 'grandeza', 'gratitud', 'inspiración', 'inspirado', 'intimidación', 'juguetón', 'logro','maravilla', 'melódico', 'motivación', 'optimismo', 'orgullo',
    'positividad', 'positivo', 'reconfortante', 'resiliencia', 'resplandor', 'reverencia', 'romance', 'satisfacción', 'serenidad','ternura', 'triunfo', 'ánimo', 'éxito','elación','éxtasis']

print(f'Sentimientos positivos: {len(positivos)}'),

# 3. SENTIMIENTOS NEGATIVOS COMPLETOS (Dolor, ira, miedo, estrés, pérdida)
negativos = ['abrumado', 'aburrimiento', 'aislamiento', 'amargura', 'angustia', 'anhelo', 'ansiedad', 'aprensivo', 'arrepentimiento', 'asco',  'decepción', 'desamor', 'desesperación', 'despectivo', 'devastado',
    'dolor', 'enojo', 'entumecimiento', 'envidia', 'envidioso', 'frustración', 'frustrado', 'lástima', 'obstáculo', 'malo', 'melancolía', 'miedo', 'negativo', 'odiar', 'pena', 'pérdida', 'reflexión', 'resentimiento', 'soledad', 'sufrimiento', 'temeroso', 'traición' , 	'tristeza' , 	'verguenza']

print(f'Sentimientos negativos: {len(negativos)}')

# 4. SENTIMIENTOS NEUTRALES (Estados ambiguos o contemplativos)
neutros =  ['ambivalencia', 'curiosidad', 'neutral','sorpresa','anticipación']
print(f'Sentimientos neutros: {len(neutros)}')

categorias = [positivos, negativos, neutros]

# Verificación del total
total_clasificados = len(positivos) + len(negativos) + len(neutros)
print(f'\n✅ Total clasificado: {total_clasificados}/{len(sentimientos_unicos_limpios)}')
print(f'   - Positivos: {len(positivos)} ({len(positivos)/106*100:.1f}%)')
print(f'   - Negativos: {len(negativos)} ({len(negativos)/106*100:.1f}%)')
print(f'   - Neutros: {len(neutros)} ({len(neutros)/106*100:.1f}%)')
print(f'Total: {len(positivos) + len(negativos) + len(neutros)}')
print()

# Verificar si existen elementos en las listas que no se encuentran en la lista sentimientos_unicos
for sentimiento in negativos + positivos + neutros:
    if sentimiento not in sentimientos_unicos_limpios:
        print(f"❌ Sentimiento no encontrado en el dataset: {sentimiento}")

# for sentimiento in negativos + positivos + neutros:
#     if sentimiento not in sentimientos_2:
#         print(f"❌ Sentimiento no encontrado en el dataset: {sentimiento}")
# Verificar si todos los sentimientos del dataset están clasificados
for sentimiento in sentimientos_unicos_limpios:
    if sentimiento not in positivos + negativos + neutros:
        print(f"❌ Sentimiento no clasificado: {sentimiento}")
# else:
#     print("✅ Todos los sentimientos del dataset están clasificados.")


Total de sentimientos únicos: 107
['abrumado', 'aburrimiento', 'aceptación', 'admiración', 'adoración', 'agradecido', 'aislamiento', 'alegría', 'amabilidad', 'amargura', 'ambivalencia', 'amistad', 'amor', 'angustia', 'anhelo', 'ansiedad', 'anticipación', 'apreciación', 'aprensivo', 'armonía', 'arrepentimiento', 'asco', 'asombro', 'cautivación', 'celebración', 'colorido', 'confiado', 'confianza', 'contentamiento', 'creatividad', 'cumplimiento', 'curiosidad', 'decepción', 'desamor', 'descubrimiento', 'desesperación', 'deslumbrar', 'despectivo', 'determinación', 'devastado', 'disfrute', 'diversión', 'dolor', 'elación', 'elegancia', 'emoción', 'empoderamiento', 'empático', 'encantamiento', 'energía', 'enojo', 'entumecimiento', 'entusiasmo', 'envidia', 'envidioso', 'esperanza', 'euforia', 'excitación', 'felicidad', 'frustración', 'frustrado', 'grandeza', 'gratitud', 'inspiración', 'inspirado', 'intimidación', 'irrelevante', 'juguetón', 'logro', 'lástima', 'malo', 'maravilla', 'melancolía', 

#### **Función para categorizar sentimientos**

In [22]:
def categorizar_sentimiento(sentimiento, categorias):
    """
    Categoriza sentimientos solo si están en las listas definidas.
    Devuelve None para sentimientos no clasificados.
    """
    sent = str(sentimiento).strip().lower()

    if sent in positivos:
        return 'positivo'
    elif sent in negativos:
        return 'negativo'
    elif sent in neutros:
        return 'neutral'
    else:
        # Devolvemos None para posterior filtrado
        return None


In [23]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
452,La euforia inunda cuando la última pieza del r...,Euforia,La euforia inunda cuando la ultima pieza del r...
165,Contemplando los misterios de la vida bajo el ...,Neutral,Contemplando los misterios de la vida bajo el ...
443,La emoción aumenta a medida que comienza la cu...,Excitación,La emocion aumenta a medida que comienza la cu...


In [24]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
256,¿Que pasaría si un capitulo de What if tempora...,negativo,¿Que pasaria si un capitulo de What if tempora...
2074,El sorprendido fui yo jajaja,neutral,El sorprendido fui yo jajaja
138,Quiero renunciar solo para tener mi liquidació...,negativo,Quiero renunciar solo para tener mi liquidacio...


In [25]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
56692,Así que intenté enseñarle a mi mamá cómo funci...,Neutral,Asi que intente enseñarle a mi mama como funci...
12558,#VALUE!,Negativo,#VALUE!
71253,@GhostRecon Casi todas las asas de las mochila...,Negativo,@GhostRecon Casi todas las asas de las mochila...


#### **Categorizar sentimientos**

In [26]:
df1_clean['Sentimiento_Final'] = df1_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df1_categorized = df1_clean[df1_clean['Sentimiento_Final'].notna()].copy()

df2_clean['Sentimiento_Final'] = df2_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df2_categorized = df2_clean[df2_clean['Sentimiento_Final'].notna()].copy()

df3_clean['Sentimiento_Final'] = df3_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df3_categorized = df3_clean[df3_clean['Sentimiento_Final'].notna()].copy()

print(f"✅ df1: {len(df1_categorized)} registros categorizados")
print(f"✅ df2: {len(df2_categorized)} registros categorizados")
print(f"✅ df3: {len(df3_categorized)} registros categorizados")

✅ df1: 732 registros categorizados
✅ df2: 2540 registros categorizados
✅ df3: 61692 registros categorizados


In [27]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
69,Al reflexionar sobre toda una vida de recuerdo...,Gratitud,Al reflexionar sobre toda una vida de recuerdo...,positivo
61,Al experimentar la emoción de una carrera de F...,Emoción,Al experimentar la emocion de una carrera de F...,positivo
206,Disfrutando de una taza de té y contemplando e...,Positivo,Disfrutando de una taza de te y contemplando e...,positivo


In [28]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
1977,Mirá el triunfo que nos llevamos en cancha de ...,neutral,Mira el triunfo que nos llevamos en cancha de ...,neutral
1891,"Medio blda esa idea de Ofe de ""que tengan mied...",neutral,"Medio blda esa idea de Ofe de ""que tengan mied...",neutral
778,Entre más reservado seas mejor te va a ir en l...,negativo,Entre mas reservado seas mejor te va a ir en l...,negativo


In [29]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
51984,¡Quedan pocos! Red Dead Redemption 2 por solo ...,Neutral,¡Quedan pocos! Red Dead Redemption 2 por solo ...,neutral
48971,Guau,Positivo,Guau,positivo
68082,"El mundo: ""La gente sigue muriendo por COVID-1...",Negativo,"El mundo: ""La gente sigue muriendo por COVID-1...",negativo


### <font color=lightgreen size=12>Limpiar dataset unificado</font>

#### **Función limpieza dataset unificado**

In [33]:
def limpiar_dataset_unificado(data, verbose=True):
    """
    Limpia dataset unificado para análisis de sentimientos.

    Proceso:
    1. Identifica y elimina CONTRADICCIONES (textos con diferentes sentimientos)
    2. Elimina DUPLICADOS exactos (mismo texto, mismo sentimiento)
    3. Limpieza final (espacios vacíos, NaN)

    Args:
        data: DataFrame con 'Texto_Limpio' y 'Sentimiento_Final'
        verbose: Si True, muestra análisis detallado

    Returns:
        DataFrame limpio, sin duplicados ni contradicciones
    """

    if verbose:
        print("🧹 LIMPIANDO DATASET UNIFICADO")
        print("-" * 50)
        print(f"Registros iniciales: {len(data):,}")
        print(f"Textos únicos iniciales: {data['Texto_Limpio'].nunique():,}")

    # Hacer copia para no modificar original
    df = data.copy()

    # ===== 1. ELIMINAR CONTRADICCIONES (PRIMERO) =====
    if verbose:
        print(f"\n1. 🔍 BUSCANDO CONTRADICCIONES...")

    # Textos con más de un sentimiento diferente
    conteo_sentimientos = df.groupby('Texto_Limpio')['Sentimiento_Final'].nunique()
    textos_con_contradiccion = conteo_sentimientos[conteo_sentimientos > 1].index.tolist()

    if textos_con_contradiccion:
        if verbose:
            print(f"   ⚠️  Encontradas: {len(textos_con_contradiccion):,} contradicciones")

            # Mostrar algunos ejemplos
            print(f"   • Ejemplos (primeros 2):")
            for texto in textos_con_contradiccion[:2]:
                sentimientos = df[df['Texto_Limpio'] == texto]['Sentimiento_Final'].unique()
                texto_corto = texto[:60] + "..." if len(texto) > 60 else texto
                print(f"     - '{texto_corto}'")
                print(f"       → Sentimientos: {', '.join(sentimientos)}")

        # Eliminar TODOS los registros de textos contradictorios
        df_sin_contradicciones = df[~df['Texto_Limpio'].isin(textos_con_contradiccion)].copy()

        if verbose:
            eliminados = len(df) - len(df_sin_contradicciones)
            print(f"   🗑️  Eliminados: {eliminados:,} registros por contradicciones")
    else:
        if verbose:
            print(f"   ✅ No hay contradicciones")
        df_sin_contradicciones = df.copy()

    # ===== 2. ELIMINAR DUPLICADOS EXACTOS =====
    if verbose:
        print(f"\n2. 🔍 BUSCANDO DUPLICADOS EXACTOS...")

    # Contar duplicados exactos (mismo texto, mismo sentimiento)
    conteo_duplicados = df_sin_contradicciones['Texto_Limpio'].value_counts()
    textos_duplicados = conteo_duplicados[conteo_duplicados > 1].index.tolist()

    if textos_duplicados:
        if verbose:
            print(f"   ⚠️  Encontrados: {len(textos_duplicados):,} textos duplicados")

            # Calcular cuántos registros se eliminarán
            total_a_eliminar = sum([conteo_duplicados[t] - 1 for t in textos_duplicados])
            print(f"   • Registros a eliminar: {total_a_eliminar:,}")

        # Eliminar duplicados (mantener primera aparición)
        df_sin_duplicados = df_sin_contradicciones.drop_duplicates(
            subset=['Texto_Limpio'],
            keep='first'
        )

        if verbose:
            eliminados = len(df_sin_contradicciones) - len(df_sin_duplicados)
            print(f"   🗑️  Eliminados: {eliminados:,} registros duplicados")
    else:
        if verbose:
            print(f"   ✅ No hay duplicados exactos")
        df_sin_duplicados = df_sin_contradicciones.copy()

    # ===== 3. LIMPIEZA FINAL =====
    if verbose:
        print(f"\n3. 🧹 LIMPIEZA FINAL...")

    df_final = df_sin_duplicados.copy()

    # Filtrar solo columnas necesarias
    df_final = df_final[['Texto_Limpio', 'Sentimiento_Final']]

    # Eliminar textos vacíos o solo espacios
    textos_vacios_antes = len(df_final)
    df_final = df_final[df_final['Texto_Limpio'].str.strip() != ""]
    textos_vacios_eliminados = textos_vacios_antes - len(df_final)

    if verbose and textos_vacios_eliminados > 0:
        print(f"   • Textos vacíos eliminados: {textos_vacios_eliminados}")

    # Eliminar sentimientos NaN
    sentimientos_nan_antes = len(df_final)
    df_final = df_final[df_final['Sentimiento_Final'].notna()]
    sentimientos_nan_eliminados = sentimientos_nan_antes - len(df_final)

    if verbose and sentimientos_nan_eliminados > 0:
        print(f"   • Sentimientos NaN eliminados: {sentimientos_nan_eliminados}")

    # ===== 4. VERIFICACIÓN Y RESUMEN =====
    if verbose:
        print(f"\n4. ✅ VERIFICACIÓN FINAL")
        print(f"   • Registros finales: {len(df_final):,}")
        print(f"   • Textos únicos finales: {df_final['Texto_Limpio'].nunique():,}")

        # Verificar que cada texto aparece solo una vez
        if len(df_final) == df_final['Texto_Limpio'].nunique():
            print(f"   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez")
        else:
            diferencia = len(df_final) - df_final['Texto_Limpio'].nunique()
            print(f"   ⚠️  ¡Problema! Hay {diferencia} duplicados")

        # Resumen
        print(f"\n" + "=" * 50)
        print("📊 RESUMEN DE LIMPIEZA")
        print("=" * 50)

        total_eliminados = (len(data) - len(df_final))
        porcentaje_eliminado = (total_eliminados / len(data)) * 100

        print(f"Registros iniciales: {len(data):,}")
        print(f"Registros finales: {len(df_final):,}")
        print(f"Total eliminados: {total_eliminados:,} ({porcentaje_eliminado:.1f}%)")

        # Distribución de sentimientos
        print(f"\n📈 DISTRIBUCIÓN FINAL DE SENTIMIENTOS:")
        distribucion = df_final['Sentimiento_Final'].value_counts()
        for sentimiento, count in distribucion.items():
            porcentaje = (count / len(df_final)) * 100
            print(f"   • {sentimiento}: {count:,} ({porcentaje:.1f}%)")

    return df_final


In [34]:
df1_categorized.sample(3)
df2_categorized.sample(3)
df3_categorized.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
6208,Definitivamente es un Walmart. A Amazon le enc...,Negativo,Definitivamente es un Walmart. A Amazon le enc...,negativo
27747,Mi carrera en D&D va viento en popa y está rea...,Positivo,Mi carrera en D&D va viento en popa y esta rea...,positivo
22271,Los políticos SIEMPRE han acusado a los buenos...,Negativo,Los politicos SIEMPRE han acusado a los buenos...,negativo


#### **Unificar datataset y limpieza**

In [35]:
print("=" * 70)
print("🔗 UNIFICANDO DATASETS CATEGORIZADOS")
print("=" * 70)

# Unificar los datasets categorizados
df_unificado = pd.concat([df1_categorized[['Texto_Limpio', 'Sentimiento_Final']], df2_categorized[['Texto_Limpio', 'Sentimiento_Final']], df3_categorized[['Texto_Limpio','Sentimiento_Final']]], ignore_index=True)

print(f"📦 Dataset unificado: {df_unificado.shape}")
print(f"   • Registros: {len(df_unificado):,}")
print(f"   • Textos únicos: {df_unificado['Texto_Limpio'].nunique():,}")


# %%
print("\n" + "=" * 70)
print("🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO")
print("=" * 70)

# Aplicar limpieza
df_final = limpiar_dataset_unificado(df_unificado, verbose=True)

🔗 UNIFICANDO DATASETS CATEGORIZADOS
📦 Dataset unificado: (64964, 2)
   • Registros: 64,964
   • Textos únicos: 57,745

🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO
🧹 LIMPIANDO DATASET UNIFICADO
--------------------------------------------------
Registros iniciales: 64,964
Textos únicos iniciales: 57,745

1. 🔍 BUSCANDO CONTRADICCIONES...
   ⚠️  Encontradas: 209 contradicciones
   • Ejemplos (primeros 2):
     - ''
       → Sentimientos: neutral, positivo, negativo
     - '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...'
       → Sentimientos: negativo, neutral
   🗑️  Eliminados: 2,633 registros por contradicciones

2. 🔍 BUSCANDO DUPLICADOS EXACTOS...
   ⚠️  Encontrados: 3,377 textos duplicados
   • Registros a eliminar: 4,795
   🗑️  Eliminados: 4,795 registros duplicados

3. 🧹 LIMPIEZA FINAL...

4. ✅ VERIFICACIÓN FINAL
   • Registros finales: 57,536
   • Textos únicos finales: 57,536
   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez

📊 RESUMEN DE LIMPIEZA
Registros

In [36]:
df_unificado.sample(3)

,Texto_Limpio,Sentimiento_Final
57761,Yo no...,negativo
35686,¡Uf! Una vez me pase al lado oscuro y elegi Ex...,neutral
44166,30. Despues de practicamente cancelar mi pedid...,negativo


 ### <font size=12 color=lightgreen>Análisis de Distribución y Visualización</font>

#### **Análisis de distribución de sentimientos**

In [37]:
#📊 ANÁLISIS DE DISTRIBUCIÓN DEL DATASET FINAL

print("=" * 60)
print("📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL")
print("=" * 60)

# 1. Calcular conteos y porcentajes
conteos = df_final['Sentimiento_Final'].value_counts()
total_registros = len(df_final)
porcentajes = (conteos / total_registros * 100).round(2)

# 2. Mostrar tabla detallada
print(f"{'SENTIMIENTO':<12} | {'CANTIDAD':>8} | {'PORCENTAJE':>10} | {'PROPORCIÓN'}")
print("-" * 50)

for sentimiento in ['positivo', 'negativo', 'neutral']:
    if sentimiento in conteos:
        count = conteos[sentimiento]
        porcentaje = porcentajes[sentimiento]
        # Crear barra visual
        barra = '█' * int(count / total_registros * 40)  # Escala a 40 caracteres
        print(f"{sentimiento.capitalize():<12} | {count:>8} | {porcentaje:>9}% | {barra}")

print("-" * 50)
print(f"{'TOTAL':<12} | {total_registros:>8} | {'100.00':>9}% | {'█' * 40}")
print("-" * 58)


📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL
SENTIMIENTO  | CANTIDAD | PORCENTAJE | PROPORCIÓN
--------------------------------------------------
Positivo     |    19220 |     33.41% | █████████████
Negativo     |    21326 |     37.07% | ██████████████
Neutral      |    16990 |     29.53% | ███████████
--------------------------------------------------
TOTAL        |    57536 |    100.00% | ████████████████████████████████████████
----------------------------------------------------------


#### **Visualización de la distribución de Sentimientos**

In [38]:
# Grafica de pastel con Plotly

valores = df_final['Sentimiento_Final'].value_counts().reset_index()
valores.columns = ['sentimientos', 'Cantidad']
fig1 = px.pie(
    names = valores.sentimientos,
    values = valores.Cantidad,
)

fig1.update_traces(textposition='inside', textinfo='label+percent',  insidetextfont=dict(color = 'white', size=14)
)

fig1.update_layout(
    title_text=f'<b>Distribución de Sentimientos</b><br><span style="font-size:14px">Dataset Final: {total_registros} registros</span>',
    title_x=0.5,
    width=500,
    height=500,
    showlegend=False,
)

fig1.show()

### <font size=12 color=lightgreen> Exportar dataset </font>

#### **Definir ruta de exportación**

In [39]:
# Ruta actual
ruta_actual = Path.cwd()

# Buscar data-science
if ruta_actual.name == 'notebooks':
    # Si estamos en notebooks/, ir a ../datasets
    carpeta_datasets = ruta_actual.parent / 'datasets'
else:
    # Buscar data-science en directorios padres
    for directorio_padre in ruta_actual.parents:
        if (directorio_padre / 'data-science').exists():
            carpeta_datasets = directorio_padre / 'data-science' / 'datasets'
            break
    else:
        # Si no encuentra, usar directorio actual/datasets
        carpeta_datasets = ruta_actual / 'datasets'

# Crear carpeta si no existe
carpeta_datasets.mkdir(parents=True, exist_ok=True)

# Ruta completa del archivo
archivo_final = carpeta_datasets / 'dataset.csv'


#### **Exportar dataset**

In [40]:
# Renombrar columnas para formato final
df_exportar = df_final.rename({
    'Texto_Limpio': 'texto',
    'Sentimiento_Final': 'sentimiento'
}, axis=1)


metadata = {
    "total_registros": len(df_exportar),
    "distribucion": dict(df_exportar['sentimiento'].value_counts()),
    "fecha_creacion": datetime.now().isoformat(),
    "version": "1.0.0",
    "fuentes": [
        "sentimentdataset_es.csv",
        "sentiment_analysis_dataset.csv"
    ]
}

# Exportar
df_exportar.to_csv(archivo_final, index=False, encoding='utf-8-sig')
print(f"✅ Dataset exportado: {archivo_final}")
print(f"📊 Registros: {len(df_exportar):,}")

# Crear copia para trabajo posterior
df = df_exportar.copy()

✅ Dataset exportado: c:\Users\marely\OneDrive\Documentos\Oracle_ONE\Hackaton\SentimentAPI-Project\sentiment-api\data-science\datasets\dataset.csv
📊 Registros: 57,536


#### **Verificar exportación**

In [41]:
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    Y verificación de integridad mejorada
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            # Probar con 5 filas primero
            df_test = pd.read_csv(ruta, encoding=enc, nrows=5)

            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                # 🔍 VERIFICACIÓN DE INTEGRIDAD MEJORADA
                print("🔍 Verificación de integridad:")
                print(f"   • Valores nulos totales: {df.isnull().sum().sum()}")
                print(f"   • Textos vacíos: {(df['texto'].str.strip() == '').sum()}")

                # Verificar que todos los sentimientos sean válidos
                sentimientos_validos = ['positivo', 'negativo', 'neutral']
                sentimientos_invalidos = df[~df['sentimiento'].isin(sentimientos_validos)]

                if len(sentimientos_invalidos) > 0:
                    print(f"   ⚠️  Sentimientos inválidos: {len(sentimientos_invalidos)}")
                    print(f"      Valores únicos inválidos: {sentimientos_invalidos['sentimiento'].unique()}")
                else:
                    print(f"   ✅ Todos los sentimientos son válidos")

                # Verificar unicidad
                textos_unicos = df['texto'].nunique()
                if len(df) == textos_unicos:
                    print(f"   ✅ 100% textos únicos: {textos_unicos:,} textos únicos")
                else:
                    print(f"   ⚠️  Duplicados: {len(df) - textos_unicos:,} textos duplicados")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df

            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

In [42]:
# Uso simple - así debería funcionar
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 57,536 registros (encoding: utf-8-sig)
🔍 Verificación de integridad:
   • Valores nulos totales: 0
   • Textos vacíos: 0
   ✅ Todos los sentimientos son válidos
   ✅ 100% textos únicos: 57,536 textos únicos
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                           texto sentimiento
    ¡Acabo de adoptar a un lindo amigo peludo!??    positivo
¡Acabo de terminar un entrenamiento increible!??    positivo


In [43]:
# Verificar que el archivo se pueda leer
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            df = pd.read_csv(ruta, encoding=enc, nrows=5)  # Probar con 5 filas
            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df
            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

# Uso simple
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 57,536 registros (encoding: utf-8-sig)
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                           texto sentimiento
    ¡Acabo de adoptar a un lindo amigo peludo!??    positivo
¡Acabo de terminar un entrenamiento increible!??    positivo


 ### <font size=12 color=lightgreen> Resumen ejecutivo </font>

In [ ]:
print("=" * 70)
print("📋 RESUMEN EJECUTIVO - HACKATHON SENTIMENT API")
print("=" * 70)
print(f"✅ Dataset final: {len(df_exportar):,} registros")
print(f"✅ Distribución balanceada: {porcentajes['positivo']}% 👍 | {porcentajes['negativo']}% 👎 | {porcentajes['neutral']}% 😐")
print(f"✅ Calidad del dataset:")
print(f"   • 0 contradicciones (cada texto tiene único sentimiento)")
print(f"   • 0 duplicados (100% textos únicos)")
print(f"   • 0 valores nulos")

📋 RESUMEN EJECUTIVO - PREPROCESAMIENTO COMPLETADO
✅ Dataset final: 57536 registros
✅ Distribución balanceada: Positivo 33.41%, Negativo 37.07%, Neutral 29.53%
✅ Archivo exportado: c:\Users\marely\OneDrive\Documentos\Oracle_ONE\Hackaton\SentimentAPI-Project\sentiment-api\data-science\datasets\dataset.csv
✅ Calidad: 0 textos vacíos, 0 valores nulos


---
### <font size=12 color=lightgreen>Observaciones</font>


### 1. **<font color='lightgreen'>Origen de los datos</font>**

Con el objetivo de mejorar la capacidad de generalización del modelo, se trabajó con dos datasets independientes obtenidos desde Kaggle.
Si bien ambos conjuntos de datos abordan el análisis de sentimiento en español, presentan diferencias en estructura, calidad lingüística y formato de origen. Su integración permitió ampliar la diversidad de expresiones textuales, reduciendo el sesgo hacia un único estilo de redacción y fortaleciendo la robustez del pipeline de preparación de datos en escenarios similares a producción.

#### **Fuentes de datos (Kaggle):**

- https://www.kaggle.com/datasets/engineercolsoquas/spanish-sentiment-analysis-dataset

- https://www.kaggle.com/datasets/kashishparmar02/social-media-sentiments-analysis-dataset

- https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis


---
### 2. **<font color='lightgreen'> Informe de Desafíos Técnicos y Soluciones</font>**

#### **Dataset** 1 – Inconsistencias en el idioma

- Problema: El dataset original presentaba traducciones incompletas, combinando registros en español con fragmentos en su idioma original, además de traducciones literales de baja calidad. Esta situación afectaba la coherencia semántica del texto y podía introducir ruido en el análisis de sentimiento.

- Solución aplicada: Se utilizó la herramienta de Traducción de Microsoft Excel como apoyo para identificar registros no traducidos. No obstante, la corrección se realizó de forma manual y supervisada, revisando y ajustando cada registro individualmente con el fin de preservar el significado original del texto y evitar distorsiones semánticas. Posteriormente, se realizó una revisión manual (sanity check) para asegurar la consistencia lingüística del dataset completo.

- Impacto en el análisis: La normalización del idioma permitió obtener un corpus coherente en español, reduciendo ambigüedades y mejorando la calidad de los datos de entrada para la etapa de clasificación de sentimiento.


**Dataset 2 – Problemas de codificación de caracteres (encoding)**

- Problema:
El segundo dataset se encontraba en formato Excel y presentaba errores de codificación al ser abierto, evidenciados por la aparición de caracteres especiales incorrectos (mojibake), lo que impedía un procesamiento adecuado del texto.

- Solución aplicada:
Como primer paso, el archivo fue exportado a formato CSV. Posteriormente, se realizó la ingesta mediante Power Query, donde se configuró explícitamente la codificación Unicode (UTF-8), corrigiendo la estructura de caracteres antes de su integración al pipeline de preparación de datos.

- Impacto en el análisis:
La corrección del encoding aseguró la correcta interpretación de caracteres propios del idioma español, evitando pérdidas de información y mejorando la calidad del texto procesado.
---


### 3. **<font color='lightgreen'>Normalización y Limpieza de Texto</font>**
- Se aplicó una función de preprocesamiento (limpiar_texto_sentimiento) que incluyó:

- Preservación de mayúsculas/minúsculas (para mantener intensidad emocional).

- Eliminación de tildes (pero conservación de ñ/Ñ).

- Limpieza de URLs, menciones y caracteres no imprimibles.

- Normalización de espacios y saltos de línea.

**Nota: Se decidió no convertir todo a minúsculas para conservar pistas contextuales (ej. “¡GENIAL!” vs. “genial”), relevantes para modelos basados en intensidad emocional.**

---

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57536 entries, 0 to 64963
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        57536 non-null  object
 1   sentimiento  57536 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


 ### <font size=12 color=lightgreen> Machine Learning</font>

In [ ]:
df.sample()

,texto,sentimiento
0,¡Acabo de adoptar a un lindo amigo peludo!??,positivo
1,¡Acabo de terminar un entrenamiento increible!??,positivo
2,¡Adoracion desbordante por un lindo cachorro r...,positivo
3,"¡Año nuevo, nuevos objetivos de fitness!??",positivo
4,¡Celebrando el cumpleaños de un amigo esta noc...,positivo
...,...,...
64959,Acabo de darme cuenta de que la particion de W...,positivo
64960,Acabo de darme cuenta de que la particion de l...,positivo
64961,Acabo de darme cuenta de que la particion de W...,positivo
64962,Acabo de darme cuenta de que entre la particio...,positivo


In [47]:
# Descargar stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

def limpiar_texto_mejorado(texto):
    """
    Limpieza de texto más conservadora que preserva palabras  de negación
    y modificadores de intensidad.
    """
    texto = texto.lower()

    # Eliminar caracteres especiales PERO preservar puntuación emocional
    texto = re.sub(r'[^a-záéíóúñ0-9\s!?.,\-]', '', texto)

    # Stopwords españolas
    stop_words = set(stopwords.words('spanish'))

    # PALABRAS CRÍTICAS A MANTENER (expandida vs. original)
    palabras_criticas = {
        # Negaciones
        'no', 'ni', 'sin', 'nunca', 'jamas', 'tampoco', 'nada', 'nadie',
        # Intensificadores
        'muy', 'mucho', 'poco', 'mas', 'menos', 'demasiado', 'bastante',
        # Modales
        'pero', 'aunque', 'sino', 'si',
        # Adjetivos de sentimiento
        'malo', 'mala', 'mal', 'bien', 'bueno', 'buena', 'mejor', 'peor',
        'horrible', 'terrible', 'excelente', 'pesimo', 'pésimo',
        # Verbos de sentimiento
        'odio', 'amo', 'encanta', 'disgusta', 'molesta',
        # Otros
        'contra', 'hacia'
    }

    # Remover stopwords EXCEPTO las críticas
    stop_words = stop_words - palabras_criticas

    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]

    return ' '.join(palabras_filtradas)

# Aplicar limpieza mejorada
df['texto'] = df['texto'].apply(limpiar_texto_mejorado)

print("✅ Limpieza de texto mejorada aplicada")
print(f"\nMuestra de textos limpios:")
print(df[['texto', 'sentimiento']].sample(5))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marely\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Limpieza de texto mejorada aplicada

Muestra de textos limpios:
                                                   texto sentimiento
56937  no encontraron asbesto laboratorios? asociaron...    negativo
16457  palabras no pueden explicar mucho odio combate...    negativo
51340  aun orgulloso ser parte de... obtener mas info...    negativo
46365  tiempo jugando red dead redemption pic.twitter...    positivo
45138  hola todos, solo pregunta, estarian mas acuerd...    positivo


In [48]:
df

,texto,sentimiento
0,acabo adoptar lindo amigo peludo!??,positivo
1,acabo terminar entrenamiento increible!??,positivo
2,adoracion desbordante lindo cachorro rescatado!??,positivo
3,"año nuevo, nuevos objetivos fitness!??",positivo
4,celebrando cumpleaños amigo noche!??,positivo
...,...,...
64959,acabo darme cuenta particion windows mac 6 año...,positivo
64960,acabo darme cuenta particion ventana mac 6 año...,positivo
64961,acabo darme cuenta particion windows mac retra...,positivo
64962,acabo darme cuenta particion windows mac tener...,positivo


 ### <font size=12 color=lightgreen> Balanceo del Dataset, TF-IDF, Modelo, Métricas y Serialización </font>

### Instalación de `imblearn`

Primero, necesitamos instalar la librería `imblearn`, que proporciona herramientas para manejar datasets desbalanceados, incluyendo la técnica SMOTE para sobremuestreo.

In [49]:
get_ipython().system('pip install imblearn')
print("Librería 'imblearn' instalada exitosamente.")

Librería 'imblearn' instalada exitosamente.


### Separación de Características y Target

Ahora, separaremos las características (el texto limpio) y la variable objetivo (el sentimiento) de nuestro DataFrame `df`. También mostraremos la distribución inicial de las clases para ver el desbalanceo.

In [50]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [51]:
# Separar X e y
X = df['texto']
y = df['sentimiento']

print("Distribución de clases:")
print(y.value_counts())

# División train/test
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain: {len(X_train_unbalanced)} | Test: {len(X_test)}")

# 🆕 TF-IDF MEJORADO
vectorizer = TfidfVectorizer(
    max_features=10000,      # ⬆ Aumentado de 5000
    ngram_range=(1, 3),      # ⬆ Trigramas (antes solo bigramas)
    min_df=2,                # 🆕 Nuevo
    max_df=0.95,             # 🆕 Nuevo
    sublinear_tf=True        # 🆕 Nuevo
)

X_train_tfidf_unbalanced = vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\n✅ Vectorización completada")
print(f"   Dimensiones: {X_train_tfidf_unbalanced.shape}")
print(f"   Vocabulario: {len(vectorizer.get_feature_names_out())} términos")

Distribución de clases:
sentimiento
negativo    21326
positivo    19220
neutral     16990
Name: count, dtype: int64

Train: 46028 | Test: 11508

✅ Vectorización completada
   Dimensiones: (46028, 10000)
   Vocabulario: 10000 términos


### <font size=12 color=lightgreen>División de Datos (Entrenamiento y Prueba) y Vectorización TF-IDF</font>



Es crucial dividir el dataset en conjuntos de entrenamiento y prueba *antes* de aplicar SMOTE para evitar la fuga de datos (data leakage). Luego, transformaremos los textos en vectores numéricos usando `TfidfVectorizer`.

In [52]:
# Dividir el dataset en conjuntos de entrenamiento y prueba ANTES de aplicar SMOTE
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTamaño del conjunto de entrenamiento (desbalanceado): {len(X_train_unbalanced)} muestras")
print(f"Tamaño del conjunto de prueba: {len(X_test)} muestras")
print(f"Distribución de clases en el conjunto de entrenamiento (desbalanceado):\n{y_train_unbalanced.value_counts()}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts()}")

# Inicializar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2)) # Limitando las características para eficiencia

# Ajustar y transformar X_train_unbalanced, y transformar X_test
X_train_tfidf_unbalanced = tfidf_vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nVectorización TF-IDF completada en la división desbalanceada.")
print(f"Forma de X_train_tfidf_unbalanced: {X_train_tfidf_unbalanced.shape}")
print(f"Forma de X_test_tfidf: {X_test_tfidf.shape}")


Tamaño del conjunto de entrenamiento (desbalanceado): 46028 muestras
Tamaño del conjunto de prueba: 11508 muestras
Distribución de clases en el conjunto de entrenamiento (desbalanceado):
sentimiento
negativo    17060
positivo    15376
neutral     13592
Name: count, dtype: int64
Distribución de clases en el conjunto de prueba:
sentimiento
negativo    4266
positivo    3844
neutral     3398
Name: count, dtype: int64

Vectorización TF-IDF completada en la división desbalanceada.
Forma de X_train_tfidf_unbalanced: (46028, 5000)
Forma de X_test_tfidf: (11508, 5000)


### <font size=12 color=lightgreen> Balanceo del Conjunto de Entrenamiento con SMOTE</font>

Ahora aplicaremos SMOTE solo al conjunto de entrenamiento vectorizado (`X_train_tfidf_unbalanced`) para balancear las clases, generando muestras sintéticas para las clases minoritarias.

In [53]:
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

print("✅ SMOTE aplicado")
print(f"\nDistribución después de SMOTE:")
print(pd.Series(y_train).value_counts())
print(f"\nNuevas dimensiones: {X_train_tfidf.shape}")

✅ SMOTE aplicado

Distribución después de SMOTE:
sentimiento
neutral     17060
positivo    17060
negativo    17060
Name: count, dtype: int64

Nuevas dimensiones: (51180, 5000)


### <font size=12 color=lightgreen> Entrenamiento del Modelo</font>


# Entrenamiento de Múltiples Modelos

Entrenaremos 3 modelos y compararemos:
1. **Logistic Regression** (baseline mejorado)
2. **SVM** (Support Vector Machine)
3. **Random Forest**

In [54]:
# 1. Logistic Regression (mejorado)
print("[1/3] Entrenando Logistic Regression...")
model_lr = LogisticRegression(
    max_iter=2000,
    random_state=42,
    C=1.0,
    solver='lbfgs',
    multi_class='multinomial'
)
model_lr.fit(X_train_tfidf, y_train)
print("✅ Logistic Regression entrenado")

# 2. SVM
print("\n[2/3] Entrenando SVM...")
model_svm = SVC(
    kernel='linear',
    C=1.0,
    probability=True,
    random_state=42,
    class_weight='balanced'
)
model_svm.fit(X_train_tfidf, y_train)
print("✅ SVM entrenado")

# 3. Random Forest
print("\n[3/3] Entrenando Random Forest...")
model_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
model_rf.fit(X_train_tfidf, y_train)
print("✅ Random Forest entrenado")

print("\n" + "="*60)
print("✅ TODOS LOS MODELOS ENTRENADOS")
print("="*60)

[1/3] Entrenando Logistic Regression...
✅ Logistic Regression entrenado

[2/3] Entrenando SVM...
✅ SVM entrenado

[3/3] Entrenando Random Forest...
✅ Random Forest entrenado

✅ TODOS LOS MODELOS ENTRENADOS


### <font size=12 color=lightgreen>Evaluación de Modelos:</font>

Evaluaremos el rendimiento del modelo en el conjunto de prueba utilizando métricas clave como accuracy, precision, recall y F1-score.

In [55]:
def evaluar_modelo(model, X_test, y_test, nombre):
    """Evaluación detallada de un modelo"""
    print(f"\n{'='*60}")
    print(f"EVALUACIÓN: {nombre}")
    print(f"{'='*60}")

    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n📊 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

    print(f"\n📋 Reporte de Clasificación:")
    print(classification_report(y_test, y_pred, digits=4))

    print(f"\n🔢 Matriz de Confusión:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    # Probabilidad promedio
    predicciones_correctas = y_pred == y_test
    probabilidades_correctas = []

    for i, correcto in enumerate(predicciones_correctas):
        if correcto:
            clase_predicha = y_pred[i]
            clase_idx = list(model.classes_).index(clase_predicha)
            probabilidades_correctas.append(y_pred_proba[i][clase_idx])

    if probabilidades_correctas:
        prob_promedio = np.mean(probabilidades_correctas)
        print(f"\n💯 Probabilidad promedio (correctas): {prob_promedio:.4f} ({prob_promedio*100:.2f}%)")

    return accuracy

# Evaluar todos los modelos
acc_lr = evaluar_modelo(model_lr, X_test_tfidf, y_test, "Logistic Regression (Mejorado)")
acc_svm = evaluar_modelo(model_svm, X_test_tfidf, y_test, "SVM")
acc_rf = evaluar_modelo(model_rf, X_test_tfidf, y_test, "Random Forest")


EVALUACIÓN: Logistic Regression (Mejorado)

📊 Accuracy: 0.7219 (72.19%)

📋 Reporte de Clasificación:
              precision    recall  f1-score   support

    negativo     0.7671    0.7604    0.7637      4266
     neutral     0.6649    0.6854    0.6750      3398
    positivo     0.7243    0.7115    0.7178      3844

    accuracy                         0.7219     11508
   macro avg     0.7188    0.7191    0.7189     11508
weighted avg     0.7226    0.7219    0.7222     11508


🔢 Matriz de Confusión:
[[3244  566  456]
 [ 484 2329  585]
 [ 501  608 2735]]

💯 Probabilidad promedio (correctas): 0.7285 (72.85%)

EVALUACIÓN: SVM

📊 Accuracy: 0.7348 (73.48%)

📋 Reporte de Clasificación:
              precision    recall  f1-score   support

    negativo     0.7737    0.7724    0.7730      4266
     neutral     0.6853    0.7010    0.6930      3398
    positivo     0.7365    0.7229    0.7297      3844

    accuracy                         0.7348     11508
   macro avg     0.7318    0.7321    

### <font size=12 color=lightgreen> Serialización del Modelo y Vectorizadors</font>



Guardaremos el modelo entrenado y el objeto `TfidfVectorizer` utilizando `joblib` para poder reutilizarlos más tarde en la API de predicción.

In [56]:
# Based on evaluation, SVM was the best model, so we'll use it for serialization.
mejor_modelo = model_svm

# Serializar el Modelo y el Vectorizador
joblib.dump(mejor_modelo, '/content/modelo_sentimientos.pkl')
joblib.dump(tfidf_vectorizer, '/content/vectorizador.pkl')

print("\nModelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/modelo_sentimientos.pkl'

In [ ]:
# Tabla comparativa
comparacion = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'SVM', 'Random Forest'],
    'Accuracy': [acc_lr, acc_svm, acc_rf]
})

comparacion['Accuracy %'] = comparacion['Accuracy'].apply(lambda x: f"{x*100:.2f}%")
comparacion['Mejora vs. Original'] = comparacion['Accuracy'].apply(lambda x: f"+{(x - 0.79)*100:.2f}%")
comparacion = comparacion.sort_values('Accuracy', ascending=False)

print("\n" + "="*70)
print("📊 COMPARACIÓN DE MODELOS")
print("="*70)
print(comparacion.to_string(index=False))
print("\n🔸 Modelo original (baseline): 79.00%")
print("="*70)

mejor_accuracy = comparacion['Accuracy'].max()
if mejor_accuracy >= 0.83:
    print("\n✅ META DE FASE 1 ALCANZADA (83-85%)")
else:
    print(f"\n⚠️ Casi alcanzado (falta {(0.83 - mejor_accuracy)*100:.2f}%)")

comparacion

### <font size=12 color=lightgreen>Pruebas con Casos Específicos</font>

Validar que ahora clasifica correctamente los casos problemáticos

In [ ]:
# Seleccionar el mejor modelo
modelos_dict = {
    'Logistic Regression': model_lr,
    'SVM': model_svm,
    'Random Forest': model_rf
}

nombre_mejor = comparacion.iloc[0]['Modelo']
mejor_modelo = modelos_dict[nombre_mejor]

print(f"🏆 Mejor modelo: {nombre_mejor}")
print(f"📊 Accuracy: {comparacion.iloc[0]['Accuracy %']}")

# Casos de prueba
casos_prueba = [
    ("mala atención", "negativo"),
    ("mal comportamiento de los empleado", "negativo"),
    ("la empresa esta perdida en lo que hace", "negativo"),
    ("pésimo servicio", "negativo"),
    ("nunca vuelvo", "negativo"),
    ("excelente servicio", "positivo"),
    ("me encantó", "positivo"),
    ("muy buena atención", "positivo"),
    ("es normal, nada especial", "neutral"),
    ("está bien", "neutral"),
]

print("\n" + "="*70)
print("PRUEBAS CON CASOS ESPECÍFICOS")
print("="*70)

aciertos = 0

for texto, esperado in casos_prueba:
    # Preprocesar
    texto_limpio = limpiar_texto_mejorado(texto)

    # Vectorizar y predecir
    texto_vectorizado = tfidf_vectorizer.transform([texto_limpio])
    prediccion = mejor_modelo.predict(texto_vectorizado)[0]
    probabilidades = mejor_modelo.predict_proba(texto_vectorizado)[0]

    clase_idx = list(mejor_modelo.classes_).index(prediccion)
    prob_prediccion = probabilidades[clase_idx]

    es_correcto = prediccion == esperado
    if es_correcto:
        aciertos += 1
        emoji = "✅"
    else:
        emoji = "❌"

    print(f"\n{emoji} '{texto}'")
    print(f"   Esperado: {esperado} | Predicho: {prediccion} | Confianza: {prob_prediccion*100:.2f}%")

print("\n" + "="*70)
print(f"RESULTADO: {aciertos}/{len(casos_prueba)} correctos ({aciertos/len(casos_prueba)*100:.1f}%")
print("="*70)

### <font size=12 color=lightgreen>Exportación del modelo</font>

In [ ]:
from sklearn.pipeline import Pipeline
import joblib

# Creamos un Pipeline manual uniendo las dos piezas
pipeline_para_produccion = Pipeline([
    ('vectorizer', tfidf_vectorizer), # Primero transforma el texto a números
    ('classifier', model)             # Luego predice con esos números
])

# Probamos que funcione antes de exportar
test_text = ["Este es un ejemplo de prueba para ver si funciona el pipeline"]
prediccion = pipeline_para_produccion.predict(test_text)
print(f"Prueba del pipeline: {prediccion}")

# EXPORTAR EL ARCHIVO FINAL
# Este es el archivo que debes subir a la carpeta de tu microservicio
joblib.dump(pipeline_para_produccion, 'modelo_entrenado.joblib')

print("✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.")

********************************************************************************************************************************************************************************************************************